In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
import time
import re

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--lang=ko_KR")

service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(10)
driver.maximize_window()

wait = WebDriverWait(driver, 10)
action = ActionChains(driver)

url = "https://www.youtube.com/watch?v=EFB18WyD1G0&list=PLWoGc25qzA3tAoQ3-7uAt7ZywMAMwbPPK"
driver.get(url)

datas = list()

playlists = driver.find_element(
    By.XPATH,
    "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[2]/div/ytd-playlist-panel-renderer/div/div[3]",
)
title_elems = playlists.find_elements(By.ID, "video-title")
for title in title_elems:
    title = title.text.split("|")[-1]
    datas.append([title])

time.sleep(3)
run_time = playlists.find_elements(By.CLASS_NAME, "badge-shape-wiz__text")
print(len(run_time))
for i, r_time in enumerate(run_time):
    r_time = r_time.text
    datas[i].append(r_time)

link_imgs = playlists.find_elements(By.CSS_SELECTOR, "img")
for i, link in enumerate(link_imgs):
    action.move_to_element(link).perform()
    link.click()

    time.sleep(1)
    try:
        premium = driver.find_element(By.XPATH,'//*[@id="dismiss-button"]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
        premium.click()
    except Exception as e:
        pass
    likes = driver.find_element(
        By.XPATH,
        '//*[@id="top-level-buttons-computed"]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button/div[2]',
    )
    likes = likes.text  # 1000 단위는 K 로 나타나 정확한 수치를 알 수 없음
    datas[i].append(likes)

    expand_button = driver.find_element(By.ID, "expand")
    expand_button.click()
    # 조회수
    views = driver.find_element(By.XPATH, '//*[@id="info"]/span[1]')
    views = int(re.sub(r"[가-힣,\s]", "", views.text))
    datas[i].append(views)
    # DATETIME
    date = driver.find_element(By.XPATH, '//*[@id="info"]/span[3]')
    date = re.sub(r"[가-힣,.:]", "", date.text).strip().split()
    for d in date:
        datas[i].append(d)

    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (
                By.XPATH,
                '//*[@id="primary-button"]/ytd-button-renderer/yt-button-shape/button',
            )
        )
    )
    action.move_to_element(button).perform()
    button.click()

    wait.until(EC.presence_of_element_located((By.ID, "secondary")))
    time.sleep(4)

    # 페이지 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 자막 세그먼트 모두 가져오기
    segments = soup.find_all(
        "div", class_="segment style-scope ytd-transcript-segment-renderer"
    )

    # 자막 시간과 텍스트 추출
    texts = list()

    for segment in segments:
        text = segment.find(
            "yt-formatted-string",
            class_="segment-text style-scope ytd-transcript-segment-renderer",
        ).get_text(strip=True)
        texts.append(text)

    datas[i].append(texts)


driver.quit()

15


In [22]:
print(datas[0])

[' 최자로드9 EP. 14', '32:11', '650', 325035, 'Sep', '12,', '2024', ['[음악] 내장이 가지고 있는 육향이 있잖아 고기의 향과 내장의 향은 다른데 내장', '본연의 향을 완전히 지우지 않고 기분좋게 즐길만큼만 남겨놨다고 그래나', '고소이 꼬리꼬리 그 꼬리꼬리 끝 때문에 한 잔 더 마시고', '싶 곱창 전골에 소주 마시고 싶을 때 서울에서는 생각나는가 그렇게 많지', '않은데 여기 되게 잘하는 중에 하나 오늘 우리는 창을 먹', '광게임 찍고온 거 같은데 오늘 오늘 게스트가 특별한 친구야 막 나도', '축구를 좀 보거든 보는 걸 되게 좋아 하는 거는 전혀 재능이 없고 거의 개발 중이 개발인데 보는 건 되게', '재밌어해서 유튜브를 보다 보면 여러 가지 채널 있잖아 근 니스타 TV 제일 잘돼 있는 거 같더라고 컨텐츠도', '엄청 많이 올라오고 퀄리티도 좋고 근데 그중에 박조민 씨라고 이렇게 보면 되게 재미있게 얘기하거든요', '축구에 대한 얘기는 그런 것들 드시는 것도 되게 좋아하시나 봐 그분 드시는 콘텐츠도 막 하시는 거 같은데 개로드', '그고 먹는 것도게 같이 재밌게 얘기해볼 수 있을 것 같아서', '야이 무슨 책이 있어 개로드 와 안녕하세요 반갑습니다 안녕하세요', '최자로드 번개 로드의 만남 또 중간에 뮤추얼 또 있어 리즈가 거 모든기를', '다보는 애같은 애들 있잖 친구인데 둘은 또 안면이 있나 봐 저희가 그', '짭짤 되잖아요 아이 뭐 아이 근데 이게 지금서 밀리고 있어 책까지 냈잖아', '책에서 내가 약간 지금 좀 지구 들어가 느낌이 있는 거야 저도 이제 그 음식 먹는 걸 좋아하니까 다른', '콘텐츠 하는 중에 어 나 여긴 이게 맛있더라 저건 이게 맛있더라 그 다 수집을 해서 인스타 계정 하나를', '만들어 놨었어요 근데 거를 저희랑 같이이라 출판사 대표님이 한번 만들어 보자고 그래서 아 근데', '만 얘기 아까워서 그럼 영상으로도 만들어보자 그렇게 해서 이제 번개로 하시고 둘은 어떻게 치지 성경이 졸

In [9]:
import pandas as pd

df = pd.DataFrame(
    datas,
    columns=["title", "run_time", "likes", "views", "year", "month", "day", "script"],
)
print(df.tail(4))

            title run_time likes   views   year month  day  \
11   최자로드9 EP. 03    32:35   709  604574  2024.    5.  30.   
12   최자로드9 EP. 02    31:42   798  310799  2024.    5.  23.   
13   최자로드9 EP. 01    29:11   924  378270  2024.    5.  16.   
14          최자로드9     1:17   257   14096  2024.    5.   9.   

                                               script  
11  [그럼 보통 몇 타코 정도해 한번 먹을 때요 정도 맛에요 정도 크기면 한 다섯 개 ...  
12  [우리의 일주일은 월 봤수 모 금금금 가득 사이 너무 때문에 실 틈 틈틈 없어졌어 ...  
13  [[음악], 여행에 즐거운 점은 언제나 그렇듯이, 새로운 데를 가보고 새로운 걸, ...  
14  [[박수], 시즌 나 기다리고, 계셨나요, 우 휴식기가, 필요한게 다시 카메라 켜도...  


In [11]:
# SAVE
df.to_csv('choiza.csv',index=False)